<table align="left">
  <td>
    <a ref="C:/Users/josep/Snowman/Reforçat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" ref="C:/Users/josep/Snowman/Reforçat.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>



In [16]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os
import random
import time
from IPython.display import clear_output


# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import glob

n=8
m=16

Mov=[0,0,0,0]

In [17]:
path="C:/Users/josep/Snowman/Dades/facils/"

files=glob.glob(path+"*.txt") 


Intern code:

- 0 : out of grid and wall (x and #)
- 1 : small ball
- 2 : medium ball
- 3 : small ball on top of a medium ball
- 4 : large ball
- 5 : small ball on top of a large ball
- 6 : medium ball on top of a large ball
- 7 : small ball on top of a medium ball on top of a large ball
- 8 : grass (')
- 9 : snow (.)
- 10: character with snow on the floor (p)
- 11: character with grass on the floor (q)


Reconpenses
- 0 moure's sense apretar 
- 0 moure's apretant bola petita
- 0 moure's apretant bola mitjana
- 0 moure's apretant bola grossa
- 100 col.locar bola mitjana sobre bola grossa
- 500 col.locar bola petita sobre boles mitjnes i grosses
- -1 Passar un instant 

Accions prohibides (-100 punts)
- sortir de la quadricula (trepitjar pared)
- fer sortir bola de la quadricula (trepitjar pared la bola)
- fer 2 boles grans
- fer dos boles mitjanes si ja tenim bola gran


Maxim episodi=50 jugades, fins acció prohibida o col.locar tres boles be

In [18]:
error=-20
tonto=-20
cami=-1
camir=0
cim=5
cimbo=20
convertir=3
bingo=50

actions=[ 
        # pared                   bola petita          bola mitjana         bola pet s mit            bola grossa           bola pet s gr         bola mit s gr             tres boles            herba                   neu              
    [[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error]], #pared
    [[None,None,None,error],[None,None,None,tonto],['pq',11,3,cim]       ,[None,None,None,tonto],['pq',11,5,cim]       ,[None,None,None,tonto],['pq',11,7,bingo]     ,[None,None,None,tonto],['pq',11,1,camir]     ,['pq',11,2,convertir]],  #bola petita
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],['pq',11,6,cimbo]     ,[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],['pq',11,2,camir]      ,['pq',11,4,convertir]],  #bola mitjana
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,2,1,-cim]       ,[None,2,1,-cim]],        #bola pet s mit
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],['pq',11,4,cami]      ,['pq',11,4,cami]],       #bola gran
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,4,1,-cim]       ,[None,4,2,-cim]],        #bola pet s gr
    [[None,None,None,error],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,None,None,tonto],[None,4,2,-cimbo]     ,[None,4,4,-cimbo]],      #bola mitj s gr
    [[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error],[None,None,None,error]], #tres boles
    [['pq',11,None,cami]   ,['pq',11,None,cami]   ,['pq',11,None,cami]   ,['pq',11,None,cami]   ,['pq',11,None,cami]   ,['pq',11,None,cami]   ,['pq',11,None,cami]   ,['pq',11,None,tonto]  ,['pq',11,None,cami]   ,['pq',11,None,cami]],    #herba
    [['pq',10,None,cami]   ,['pq',10,None,cami]   ,['pq',10,None,cami]   ,['pq',10,None,cami]   ,['pq',10,None,cami]   ,['pq',10,None,cami]   ,['pq',10,None,cami]   ,['pq',10,None,tonto]  ,['pq',10,None,cami]   ,['pq',10,None,cami]]     #neu
] 




In [19]:


ree="x#'.pq1234567"
reem=[0,0,8,9,10,11,1,2,3,4,5,6,7]
def lleguir_tauler(nom,n=n,m=m):
    f = open(nom, "r")
    tauler=np.zeros((n,m),dtype=int)
    for row,linea in enumerate(f):
        linea=linea.rstrip('\n\r\t')
        for column,car in enumerate(linea):
            res = ree.find(car)
            tauler[row,column]=reem[res]
    f.close()
    return np.array(tauler)



In [20]:
def mod_step(t,reward,row,column,sega,segb,segaa,segbb):
    bonus=2
    if reward==cami:
        ball=[1,2]
        distact=distancia(t,row,column,ball)
        distnou=distancia(t,sega,segb,ball)
        if distnou<distact:
            reward=cami+bonus
    if reward==camir:
        if t[sega,segb]==1:
            ball=[2,6]
        elif t[sega,segb]==2:
            ball=[4]
        else:
            ball=[]
        distact=distancia(t,row,column,ball)
        distnou=distancia(t,sega,segb,ball)
        if distnou<distact:
            reward=camir+bonus
    if reward==convertir:
        if t[sega,segb]==2:
            res = np.isin(t,[4,5,6])
            res = np.array(np.where(res))
            if np.size(res)!=0:
                reward=error
        else:
            res = np.isin(t,[2,3,6])
            res = np.array(np.where(res))
            res2= np.isin(t,[4,5,6])
            res2= np.array(np.where(res2))
            if np.size(res)>0:
                if res.shape[1]>=2 or (res.shape[1]==1 and res2.shape[1]>=1):
                    reward=error
    
        
    return reward



In [21]:
def distancia(t,r,c,ball):
    dist=500
    for b in ball:
        res = np.array(np.where(t == b))
        if np.size(res)!=0:
            for j in range(res.shape[1]):
                if abs(r-res[0][j])+abs(c-res[1][j])<dist:
                    dist=abs(r-res[0][j])+abs(c-res[1][j])
    return dist



def step(t,action,passat): # 0 dreta, 1 baix, 2 esquerra, 3 dalt
    snow=True
    res = np.array(np.where(t == 10))
    if np.size(res)==0: 
        snow=False
        res = np.array(np.where(t == 11))
    a=res[0][0]
    b=res[1][0]
    
    inc=[[0,1,0,-1],[1,0,-1,0]]
    seg=[a+inc[0][action],b+inc[1][action]]
    seg2=[a+2*inc[0][action],b+2*inc[1][action]]

    Mov[action]=Mov[action]+1
    if seg2[0]<0 or seg2[0]>=8 or seg2[1]<0 or seg2[1]>=16:
        aux=0
    else:
        aux=t[seg2[0],seg2[1]]

    mact=actions[t[seg[0],seg[1]]][aux]
    reward=mact[3]
    mact=mact[:3]
    for i,aux in enumerate(mact):
        if aux!=None:
            if aux=='pq':
                if snow:
                    f=9
                else:
                    f=8
            else:
                f=int(aux)

            if i==0:
                t[a,b]=f
            elif i==1:
                t[seg[0],seg[1]]=f
            else:
                t[seg2[0],seg2[1]]=f

    if reward==cami or reward==camir: #no pot tornar a passar per el mateix lloc
        passat[seg[0],seg[1]]+=1
        if passat[seg[0],seg[1]]>1:
            reward=tonto
    else:
        passat=np.zeros((n,m))
    
    if reward==bingo or reward<=tonto:
        done=True
    else:
        done=False

    return (t,reward,done,a,b,seg[0],seg[1],seg2[0],seg2[1],passat)



In [22]:
def arreglar_taula(t):
    t2=t.copy()
    res = np.array(np.where(t2 == 11))
    if np.size(res)!=0:
        t2[res[0][0],res[1][0]]=10
    return t2/10.

In [23]:

from random import choices



def play_one_step(t, model, loss_fn,iteration,passat):
    with tf.GradientTape() as tape:
        left_proba = tf.squeeze(model(arreglar_taula(t)[np.newaxis]))
    
        r=np.random.uniform()
        rati=max((1000-iteration)/1000*0.6,0.10)
        if r>rati:
            action = choices(population=range(4), k=1, weights=left_proba)
        else:
            action = choices(population=range(4), k=1)
       
        y_target = np.eye(4)[action[0]]
        loss = tf.reduce_mean(loss_fn(y_target, left_proba))
    grads = tape.gradient(loss, model.trainable_variables)
    t, reward, done, a, b, sega, segb, segaa, segbb, passat= step(t,action[0],passat)
    reward=mod_step(t,reward,a,b,sega,segb,segaa,segbb)
    return t,reward, done, grads, rati, passat



In [24]:
def play_multiple_episodes(n_episodes, n_max_steps, model, loss_fn,iteration):
    all_rewards = []
    all_grads = []

    
    nom=files[iteration % len(files)]
    tauler=lleguir_tauler(nom,n=n,m=m)
    for i in range(4):
        maxim=-1000000
        for episode in range(n_episodes):
            t=tauler.copy()
            passat=np.zeros((n,m))

            antact=-3
            reward=-1
            current_rewards = []
            current_grads = []
            for step in range(n_max_steps):
                t, reward, done, grads, rati, passat = play_one_step(t, model, loss_fn,iteration,passat)
                current_rewards.append(reward)
                current_grads.append(grads)
                if done:
                    break
            if sum(current_rewards)>maxim:
                maxim=sum(current_rewards)
            all_rewards.append(current_rewards)
            all_grads.append(current_grads)
        if i%2==0:
            tauler=np.flip(tauler,0)
        else:
            tauler=np.flip(tauler,1)
        
    return all_rewards, all_grads, maxim, nom, rati



In [25]:
def discount_rewards(rewards, discount_rate):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_rate
    return discounted

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate)
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean) / reward_std
            for discounted_rewards in all_discounted_rewards]

In [26]:
n_iterations = 2500
n_episodes_per_update = 25
n_max_steps = 50
discount_rate = 0.50

In [27]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.categorical_crossentropy

In [28]:
tf.keras.backend.clear_session()
model=tf.keras.models.load_model('C:/Users/josep/Snowman/snowv1-10-f.h5')
#model.load_weights('C:/Users/josep/Snowman/Dades/jugades/checkpoint/checkpoint')
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8, 16)]           0         
_________________________________________________________________
reshape (Reshape)            (None, 8, 16, 1)          0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 16, 32)         320       
_________________________________________________________________
dropout (Dropout)            (None, 8, 16, 32)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 8, 32)          0         
_________________________________________________________________
batch_normalization (BatchNo (None, 4, 8, 32)          128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 8, 64)          18496 

In [29]:
def prova(nom=None):
    if nom==None:
        nom=files[random.randint(0,len(files)-1)]
    
    tauler=lleguir_tauler(nom,n=n,m=m)
    print(tauler)
    time.sleep(1)
    clear_output(wait=True)
    sum_rewards=0

    for i in range(30):
        action2=model.predict(arreglar_taula(tauler)[np.newaxis]).squeeze()
        action=np.argmax(action2)
        clear_output(wait=True)
        tauler, reward, done, a, b, sega, segb, segaa, segbb, passat= step(tauler,action,np.zeros((n,m)))
        sum_rewards+=reward
        print(action,i,sum_rewards,action2,done)
        print(tauler)
        time.sleep(5)
        if done:
            break
        clear_output(wait=True)

In [31]:


estadistiques=[]
for i in range(len(files)):
    estadistiques.append([os.path.basename(files[i]),-2000,-2000,-1,-2000,-2000,-1])

for iteration in range(n_iterations):
    if (iteration+1)%100==0:
        prova(nom='C:/Users/josep/Snowman/Dades/facils/david-11.txt')
    
    all_rewards, all_grads, maxim, nom, rati= play_multiple_episodes(n_episodes_per_update, n_max_steps, model, loss_fn,iteration)

    total_rewards = sum([sum(ar) for ar in all_rewards])                   # Not shown in the book

    [nom,actmean,maxmean,itmean,actmaxim,maxmaxim,itmaxim]=estadistiques[iteration%len(files)]

    actmean=total_rewards/(n_episodes_per_update*4)
    if actmean>=maxmean:
        maxmean=actmean
        itmean=iteration
    actmaxim=maxim
    if actmaxim>=maxmaxim:
        maxmaxim=actmaxim
        itmaxim=iteration
    estadistiques[iteration%len(files)]=[nom,actmean,maxmean,itmean,actmaxim,maxmaxim,itmaxim]
    clear_output(wait=True)
    print("\r**** Iteration: {} mean rewards: {:6.1f} mov: {} maxim: {} nom: {:12} rati: {:.3f}  ".format(iteration, total_rewards/(n_episodes_per_update*4),Mov,maxim,nom,rati), end="") # Not shown
    
    for element in estadistiques:
        [nom,actmean,maxmean,itmean,actmaxim,maxmaxim,itmaxim]=element
        print("\n\rnom: {:12} mean: {:8.1f} {:8.1f} {:4}     max: {:6} {:6} {:4}   ".format(nom,actmean,maxmean,itmean,actmaxim,maxmaxim,itmaxim), end="")
   
   
    all_final_rewards = discount_and_normalize_rewards(all_rewards, discount_rate)
    
    all_mean_grads = []
    for var_index in range(len(model.trainable_variables)):
        mean_grads = tf.reduce_mean(
            [final_reward * all_grads[episode_index][step][var_index]
             for episode_index, final_rewards in enumerate(all_final_rewards)
                 for step, final_reward in enumerate(final_rewards)], axis=0)
        all_mean_grads.append(mean_grads)
        

        
    optimizer.apply_gradients(zip(all_mean_grads, model.trainable_variables))




**** Iteration: 1735 mean rewards:  -19.5 mov: [335143, 422878, 378948, 457216] maxim: -14 nom: david-23.txt rati: 0.100  
nom: david-1.txt  mean:    -11.5     -6.8  840     max:    -14      3  651   
nom: david-10.txt mean:    -11.7     -7.0 1660     max:     55     55 1723   
nom: david-11.txt mean:    -20.9    -18.9 1325     max:    -18     52 1577   
nom: david-12.txt mean:    -22.0    -21.0 1557     max:    -20     50 1704   
nom: david-14.txt mean:    -22.1    -21.4 1495     max:    -20    -20 1726   
nom: david-15.txt mean:    -26.9    -24.5  236     max:    -20    -20 1727   
nom: david-16.txt mean:    -19.9    -17.1 1581     max:      0      1 1644   
nom: david-17.txt mean:     -4.8     -2.3 1561     max:      2      2 1729   
nom: david-18.txt mean:    -12.9    -10.5 1667     max:    -13      4 1667   
nom: david-19.txt mean:     -6.1     -1.5 1626     max:      0      2 1689   
nom: david-20.txt mean:    -20.0    -15.8 1123     max:    -13    -11  199   
nom: david-21.txt m

In [ ]:

prova(nom='C:/Users/josep/Snowman/Dades/facils/david-18.txt')





model.save('snowv1-11-f.h5')
